In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
%matplotlib inline

# 이미지 로드 함수 정의

In [2]:
def imgLoad(imgName) :
    return mpimg.imread(imgName)

# 그레이 이미지 생성

In [3]:
def getGray(img) :
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# 가우시안 블러 처리

In [4]:
def gaussian_blur(img) :
    kernel_size = 5
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

# 캐니 엣지 검출 알고리즘 적용

In [5]:
def canny(blur_gray) :
    low_threshold = 50
    high_threshold = 100
    return cv2.Canny(blur_gray, low_threshold, high_threshold)

# 관심영역 설정

In [12]:
def setVertices(img) :
    imshape = img.shape
    vertices = np.array([[(0, imshape[0]),
                            (240, 230),
                            (480, 230),
                            (imshape[1], imshape[0])]], dtype=np.int32)
    return vertices

# 관심영역을 이미지에 반영

In [13]:
def maskedImg(img, vertices) :
    mask = np.zeros_like(img)
    
    if len(img.shape) > 2 :
        channel_count = img.shape[2]
        ignore_mask_color = (255, ) * channel_count
    else :
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    return cv2.bitwise_and(img, mask)

# 라인 생성 정의

In [8]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=5) :
    left = False
    right = False
    if lines is None :
        return
    for line in lines :
        for x1, y1, x2, y2 in line :
            if (x1 - x2) / (y1 - y2) < 0 :
                if left == False :
                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                    left = True
            elif (x1 - x2) / (y1 - y2) > 0 :
                if right == False : 
                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                    right = True

# 차선 검출

In [9]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap) :
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                           minLineLength=min_line_len,
                           maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# 본래의 영상과 합성

In [10]:
def getMixed(img, initial_img, a = 0.8, b = 1., c = 0.) :
    return cv2.addWeighted(initial_img, a, img, b, c)

# 메인 함수

In [14]:
from IPython.display import clear_output
if __name__ == "__main__" :
    for i in range (1, 750) :
        imgName = 'Lane1\\' + str(i).zfill(5) + '.jpg'
        img = imgLoad(imgName)
        gray = getGray(img)
        blur_gray = gaussian_blur(gray)
        edges = canny(blur_gray)
        vertices = setVertices(edges)
        mask = maskedImg(edges, vertices)
        
        rho = 1
        theta = np.pi/180
        threshold = 80
        min_line_len = 100
        max_line_gap = 250
        
        lines = hough_lines(mask, rho, theta, threshold, min_line_len, max_line_gap)
        result = getMixed(lines, img)
        
        plt.figure(figsize=(10, 8))
        plt.imshow(result, cmap='gray')
        plt.show()
        clear_output(wait = True)

KeyboardInterrupt: 